In [2]:
import keras
import csv
import numpy as np

from tensorflow.keras import Sequential, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

2022-03-15 16:25:27.680460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-15 16:25:27.680481: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
def import_data():
    return np.genfromtxt('data_kia.csv', delimiter=',')

def normalize_data(rows):
    max_columns = rows.max(axis=0)
    max_columns = rows.min(axis=0)
    for i in rows[0].size:
        for j in rows.size:
            xmax = max_columns[j]
            xmin = min_columns[j]

In [ ]:
def create_model(learning_rate=1e-4, window_size=16, num_features=51):
    model = Sequential()
    model.add(LSTM(160, input_shape=(window_size, num_features)))
    model.add(LSTM(200, ))

    model.add(Dense(10, activation="sigmoid"))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="categorical_crossentropy", metrics=['categorical_crossentropy'])

    return model